# Download the buildings data

Question: How many buildings were mapped with RapID (Microsoft buildings) in selected region?

#### Import libs

In [7]:
import json
import os
import requests

### Get the data for every specified region

#### Definitions

In [8]:
# define the dictinaries

data_dic = {0: "blds_continent_data_18-23_ai.json",
            1: "blds_africa_states_18-23_ai.json",
            2: "blds_northamerica_states_18-23_ai.json",
            3: "blds_europe_states_18-23_ai.json",
            4: "blds_asia_states_18-23_ai.json", }

geojson_folder_dic = {0: "geojson-continent-states",
                      1: "geojson-africa-states",
                      2: "geojson-northamerica-states",
                      3: "geojson-europe-states",
                      4: "geojson-asia-states",}

In [9]:
# define the user input

while True:
    try:
        user_d_spec = int(input("Which data do you want to download?\
                            Enter 0 for continents (2018–2023),\
                            1 for Africa states (2018–2023),\
                            2 for North-Central America states (2018–2023)\
                            3 for Europe states (2018–2023)\
                            4 for Asia states (2018–2023)"
                                ))
    except ValueError:
        print("Please enter a number.")
        continue
    if user_d_spec not in range(len(geojson_folder_dic)):
        print("Please enter 0, 1, 2, 3 or 4.")
        continue
    break


In [10]:
# define the functions


def data_gen(geojson_folder_dic, name):

    with open(f"geojson-regions\\{geojson_folder_dic}\\{name}", "r") as file:
        bpolys = json.load(file)

    parameters = {
        "bpolys": json.dumps(bpolys),  # pass GeoJSON as string.
        "filter": "building=* and building!=no and geometry:polygon",
        "groupByKey": "source",
        "groupByValues": "microsoft/BuildingFootprints",
        "format": "json",
        "time": "2018-01-01/2023-01-01/P1Y",
    }
    headers = {
        "accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded",
    }

    for value in parameters.values():
        assert value != "", "Please provide values for the parameters"

    response = requests.post(url, data=parameters, headers=headers)
    response.raise_for_status()  # Raise an Exception if HTTP Status Code is not 200

    print("Response:")
    print(json.dumps(response.json(), indent=4))  # Pretty print response

    result = response.json()["groupByResult"]
    return result


def get_geojson_names(geojson_folder_dic):

    os.chdir('..')
    home_dir = os.getcwd()
    geojson_dir = os.path.join(home_dir, f"geojson-regions\\{geojson_folder_dic}", "")

    geojson_names = []
    for (dir_path, dir_names, file_names) in os.walk(geojson_dir):  # give the path
        geojson_names.extend(file_names)
    print(geojson_names)
    return geojson_names


In [11]:
# define the URL

base_url = "https://api.ohsome.org/v1"
endpoint = "/elements/count/groupBy/boundary/groupBy/tag"
url = base_url + endpoint

#### Get the data

In [6]:
data = []

for geojson_name in get_geojson_names(geojson_folder_dic[user_d_spec]):

    data.append(data_gen(geojson_folder_dic[user_d_spec], geojson_name))
    # print(geojson_name)


['0_Indonesia.geojson', '10_Bhutan.geojson', '11_Oman.geojson', '12_Uzbekistan.geojson', '13_Kazakhstan.geojson', '14_Tajikistan.geojson', '15_Mongolia.geojson', '16_Russia.geojson', '17_Vietnam.geojson', '18_Cambodia.geojson', '19_United Arab Emirates.geojson', '1_Malaysia.geojson', '20_Georgia.geojson', '21_Azerbaijan.geojson', '22_Turkey.geojson', '23_Laos.geojson', '24_Kyrgyzstan.geojson', '25_Armenia.geojson', '26_Iraq.geojson', '27_Iran.geojson', '28_Qatar.geojson', '29_Saudi Arabia.geojson', '2_Cyprus.geojson', '30_Pakistan.geojson', '31_Thailand.geojson', '32_Kuwait.geojson', '33_Timor-Leste.geojson', '34_Brunei.geojson', '35_Myanmar.geojson', '36_Bangladesh.geojson', '37_Afghanistan.geojson', '38_Turkmenistan.geojson', '39_Jordan.geojson', '3_India.geojson', '40_Nepal.geojson', '41_Yemen.geojson', '42_N. Cyprus.geojson', '43_Cyprus U.N. Buffer Zone.geojson', '44_Siachen Glacier.geojson', '45_Philippines.geojson', '46_Sri Lanka.geojson', '47_Taiwan.geojson', '48_Japan.geojson',

In [7]:
# export the data

def export_data(name):
    with open(f"downloaded-data\\{name}", "w") as file:
        json.dump(data, file, indent=4)


export_data(data_dic[user_d_spec])
